In [3]:
# Download libraries
import requests
import pandas as pd
import numpy as np
import io
from datetime import date
from datetime import timedelta

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
# Maintenance reports download
def download(file, url):
    print('\nGetting data...')
    try:
        response = requests.request("GET", url)
        with open('path'+ file, 'w', encoding="utf-8") as f:
            f.write(response.text)
        print('Data was updated at', datetime.now())
        print('Continue with the next file.')
    except:
        ("\nAn error occured with download.")

In [ ]:
print('*'*10,'Item Maintenance Report','*'*10)
url = input ('Please enter the download URL for the maintenance report: ')
try:
    response = requests.request("GET", url)
    rawData = pd.read_csv(io.StringIO(response.text))

    
    #Get date
    daysago = date.today() - timedelta(days = 14)
    
    #Filter non-ShoutOut
    final_df = rawData[(~rawData['Item Type'].str.contains('ShoutOut')) & 
                        (rawData['Users Interacted'] == 0) &
                        ((rawData['Users Exposed'] >= 15) | (rawData['Users Exposed] == 0)) &
                        (rawData['Item Last Change Date'] <= daysago.strftime("%Y-%m-%d")) &
                        (~rawData['System Name'].isin(['System1', 'System2'])) &
                        (~rawData['Item Name'].str.contains('Asterisk')) &
                        (~rawData['Item Name'].str.contains('Highlight'))]
    
    final_df.to_csv('path/file.csv', index=False)
except:
    ("\nAn error occured with download.")
print('*'*20, 'Complete', '*'*20)

In [ ]:
print('*'*10,'Menu Searches','*'*10)
url = input ('Please enter the download URL for the menu searches report: ')
try:
    response = requests.request("GET", url)
    rawData = pd.read_csv(io.StringIO(response.text))
    
    final_df = rawData[(rawData['Menu Searches'] >= 5) &
            (rawData['Menu Searches Resulted in Action']/rawData['Menu Searches'] <= .4) &
            (rawData['Search Term'] != 'term')]
    
    final_df.to_csv('path/file.csv', index=False)
    
except:
    ("\nAn error occured with download.")
print('*'*20, 'Complete', '*'*20)

In [ ]:
print('*'*10,'Error Report','*'*10)
url = input ('Please enter the download URL for the errors report: ')

try:
    response = requests.request("GET", url)
    rawData = pd.read_csv(io.StringIO(response.text))
    
    # Get SWTs with > 2 plays and > 30% failure rate
    # find total plays
    plays = rawData.groupby(['ID', 'Failed Step Index'], as_index=False)['Total Plays'].sum()
    # Find average failures
    per_failure = rawData.groupby(['ID', 'Failed Step Index'], as_index=False)['% Step Failures From All Plays'].max()
    # Join
    df = plays.join(per_failure.set_index(['ID', 'Failed Step Index']), on=['ID', 'Failed Step Index'])
    # Filter
    filtered_df = df[(df['Total Plays']>2) & (df['% Step Failures From All Plays']>=33)]
    
    
    # Find yesterday's data
    yesterday = date.today() - timedelta(days = 1)
    yesterday_df = rawData[(rawData['Play Date'] == yesterday.strftime("%Y-%m-%d")) &
                           (rawData['% Step Failures From All Plays']>=20)]
    
    # Convert for final record filtering
    final_df = filtered_df[filtered_df['ID'].isin(yesterday_df['ID']) & filtered_df['Failed Step Index'].isin(yesterday_df['Failed Step Index'])]
    df_response = rawData[rawData['ID'].isin(final_df['ID'])& rawData['Failed Step Index'].isin(yesterday_df['Failed Step Index'])]
    
    df_response.to_csv('path/file.csv', index=False)
            
except:
    ("\nAn error occured with download.")
print('*'*20, 'Complete', '*'*20)